In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
%matplotlib inline
import plotly.express as px
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tools.eval_measures import rmse, aic
import warnings
warnings.simplefilter('ignore')
import joblib
import os

In [30]:
df=pd.read_csv('output.csv')
mpe=pd.read_csv('Mapescoreinfo.csv')

state_dist=set(zip(df['State'],df['District']))
df['Date']=pd.to_datetime(df['Date'])
df=df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

def mape(y2, y_pred): 
    y2, y_pred = np.array(y2), np.array(y_pred)
    return np.mean(np.abs((y2 - y_pred) / y2)) * 100


def plotres(predicted_data,test):
    f,ax=plt.subplots()
    plt.plot(predicted_data,c='green', label= 'predictions')
    plt.plot(test, c='red',label='real values')
    #X_test['Confirmed'].values
    plt.legend()
    plt.title('True vs predicted values')

i=0

for [s,d] in state_dist:
    modelname="polymodel_for_state_"+s+"_district_"+d
    if modelname in os.listdir():
        continue
    else:     
        try:
            dftemp=df[df['District']==d]
            dftemp= dftemp.set_index(['Date'])
            # Quality the parameter to be predicted is represented as X.
            X = dftemp[['Confirmed']]
            # All the input parameters used to predict the value are represented as y.
            y = dftemp[['Tested','Age_Group_0_29','Age_Group_30_49','Age_Group_50']]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            # Fit the training data into polynomial model of degree 4 (appropriate value as per the data-set).
            model = PolynomialFeatures(degree=4)
            y_ = model.fit_transform(y_train)
            y_test_ = model.fit_transform(y_test)

            # Fit and predict the obtained polynomial model into linear regression.
            lg = LinearRegression()
            lg.fit(y_,X_train)
            joblib.dump(lg,modelname)
            m=joblib.load(modelname)
            X_test['pred']=m.predict(y_test_)
            
            vl=mape(X_test['Confirmed'],X_test['pred'])
            print(vl)
            mpe.at[mpe['District']==d,'MapeScore poly']=vl
        except:
            continue



25.442978369041313
18.072688536181545
8.386797439207387
inf
31.67734039764307
7.07304135826725
8.16656051846525
inf
90.06853447594149
35.80602062255675
80.50003488162628
5.361445731124503
4.778476783865664
6.95439532689204
9.614444021914085
2.8631439500207105
261855.2650924148
3.3292799174175975
70.8370046603859
6.692397264427775
15.665254079433272
4.537304229157245
4.57507743179787
inf
8.28386744407086
17.583538787408084
4.588432886209501
50.92026207910034
1.4123881275245393
100.0
3.0984421023615196
3.504762390865731
6.018772940582853
17.160695310987123
nan
244.00435607326904
118.73671690948717
13.1117266485584
44.60079824632162
62165948.37724637
7.553330007629119
718.8552894679514
39.096496398093336
302.40505248080456
32.85854436802891
20.23195436260093
191.92393998259737
13.966126267047613
7.701312571979418
15.846373886317036
27.104089050714613
1.3110006431027363
16.5989394659978
inf
inf
51.34053162041483
25.60411828233286
1.5740776652371617
57.353579745544714
34.06201969331319
33.6

In [35]:
# mpe=mpe[~mpe.isin([np.nan, np.inf, -np.inf]).any(1)]
mpe.to_csv('mpescorefinl.csv')

In [36]:
mpe

Unnamed: 0  Unnamed: 0.1          State        District  MapeScore arima  \
0             0             0          Bihar          Munger        11.362465   
2             2             2     Tamil Nadu    Kancheepuram         9.737995   
3             3             3     Tamil Nadu  Tiruvannamalai         7.140038   
8             8             8  Uttar Pradesh         Mathura         2.227698   
12           12            12        Gujarat          Rajkot         6.878831   
..          ...           ...            ...             ...              ...   
456         456           456         Punjab        Ludhiana        16.775813   
457         457           457      Rajasthan          Nagaur         7.675547   
460         460           460         Punjab      Hoshiarpur        37.418586   
461         461           461          Bihar        Vaishali         8.245734   
469         469           469      Rajasthan           Dausa        12.545968   

     MapeScore var  MapeScore poly  
0        10.980605    2.127327e+00  
2         7.188084    1.200296e+06  
3         1.883094    3.580602e+01  
8        10.410882    5.594227e+00  
12        6.706756    2.816270e+01  
..             ...             ...  
456      10.906830    3.629852e+01  
457       2.647443    3.741956e+00  
460      47.661618    3.664451e+01  
461       4.898353    8.187583e+00  
469       2.083951    9.569445e+00  

[190 rows x 7 columns]